In [1]:
DRIVE=False
root_folder = ""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_folder = "/content/drive/MyDrive/CS182-Spring2020-NLP-Project/"

Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install segtok

  Created wheel for segtok: filename=segtok-1.5.10-cp37-none-any.whl size=25019 sha256=47a7d835dc043e44b8199ed2b037303977dde306614277aec023b58819b7d881
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully built segtok


In [4]:
import os
import sys
sys.path.append(root_folder)
from segtok import tokenizer
from collections import Counter
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import json
import textwrap

In [5]:

# Dataset has 533581 reviews
reviews_train = []
ratings_train = []
reviews_test = []
ratings_test = []

SPLIT_RATIO = 0.7
DATASET_SIZE = 533581
TRAIN_SIZE = round(DATASET_SIZE * SPLIT_RATIO)

with open(root_folder+"yelp_review_training_dataset.jsonl", "r") as f:

    count = 0

    for count, line in enumerate(f):
        #review = textwrap.shorten(json.loads(line)['text'], width=100)
        #### CROP first 130 characters #####
        review = json.loads(line)['text'][:130]
        rating = int(json.loads(line)['stars']) - 1
        if count <= TRAIN_SIZE:

            reviews_train.append(review)
            ratings_train.append(rating)
        else:

            reviews_test.append(review)
            ratings_test.append(rating)

print("Training Set Size: " + str(len(reviews_train)))
print("Test Set Size: " + str(len(reviews_test)))
	##print(ratings)

Training Set Size: 373508
Test Set Size: 160073


In [ ]:
#########  DO NOT RUN THIS CELL JUST EXPERIMENTAL ########
#save dataset to txt file
np_training_review = np.array(reviews_train)
np_training_rating = np.array(ratings_train)
np_test_review = np.array(reviews_test)
np_test_rating = np.array(ratings_test)

np.savetxt(root_folder+'train_review.txt', np_training_review, delimiter=" ", fmt="%s")
np.savetxt(root_folder+'test_review.txt', np_test_review, delimiter=" ", fmt="%s")
np.savetxt(root_folder+'train_rating.txt', np_training_rating, delimiter=" ", fmt="%s")
np.savetxt(root_folder+'test_rating.txt', np_test_rating, delimiter=" ", fmt="%s")

In [6]:
# Print DATA to check
for i in range(5):
    print("Review " + str(i)+": ")
    print(reviews_train[i])
    print(ratings_train[i])

Review 0: 
Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked onlin
0
Review 1: 
I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains t
4
Review 2: 
I have to say that this office really has it together, they are so organized and friendly!  Dr. J. Phillipp is a great dentist, ve
4
Review 3: 
Went in for a lunch. Steak sandwich was delicious, and the Caesar salad had an absolutely delicious dressing, with a perfect amoun
4
Review 4: 
Today was my second out of three sessions I had paid for. Although my first session went well, I could tell Meredith had a particu
0


In [ ]:

##### DONT RUN THIS CELL EXPERIMENTAL ######
# Try using nltk
!pip install nltk
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

reviews_train_shorten = []
print(len(reviews_train))
for i, sentence in enumerate(reviews_train):
    
    stop_words = set(stopwords.words('english')) 
    
    word_tokens = word_tokenize(sentence) 
    
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    new_sentence = " "

    new_sentence = new_sentence.join(filtered_sentence)

    reviews_train_shorten.append(new_sentence)

    if i % 10000 == 9999:
        print(i)
print("Modified Sentences: ")
for i in range(5):
    print(reviews_train_shorten[i])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
373508
9999
19999


KeyboardInterrupt: ignored

In [ ]:
##### DONT RUN THIS CELL EXPERIMENTAL ######
reviews_test_shorten = []
for i, sentence in enumerate(reviews_test):
    
    stop_words = set(stopwords.words('english')) 
    
    word_tokens = word_tokenize(sentence) 
    
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    new_sentence = " "

    new_sentence = new_sentence.join(filtered_sentence)

    reviews_test_shorten.append(new_sentence)

    if i % 10000 == 9999:
        print(i)
print(len(reviews_test_shorten))
print("Modified Sentences: ")
for i in range(5):
    print(reviews_test_shorten[i])


9999
19999
29999
39999
49999
59999
69999
79999
89999
99999
109999
119999
129999
139999
149999
159999
160073
Modified Sentences: 
This truly excellent doctor office needs major help . The doctor really cares
Visiting Vancouver , mostly Japanese ramen , place outstanding . Chinese fla
Went day family meal five us $ 20 leftovers . It clean
I 've tried place times I never learn . They set sports bar ,
$ 30 entrance fee ? ? ? I n't get . The samples miniscule . It reall


In [13]:
!pip install transformers

In [8]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased', num_labels=5)

In [10]:

train_encodings = tokenizer(reviews_train, truncation=True, padding=True)
test_encodings = tokenizer(reviews_test, truncation=True, padding=True)

#train_encodings = tokenizer(reviews_train_shorten, truncation=True, padding=True)
#test_encodings = tokenizer(reviews_test_shorten, truncation=True, padding=True)

In [11]:

class YelpDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = YelpDataset(train_encodings, ratings_train)
test_dataset = YelpDataset(test_encodings, ratings_test)

In [12]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=5)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=1e-4)

TRAINING_EPOCH = 1
for epoch in range(TRAINING_EPOCH):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

model.eval()

cuda


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

[1,  1000] loss: 0.980
[1,  2000] loss: 0.913
[1,  3000] loss: 0.896
[1,  4000] loss: 0.886
[1,  5000] loss: 0.859
[1,  6000] loss: 0.860
[1,  7000] loss: 0.864
[1,  8000] loss: 0.864
[1,  9000] loss: 0.858
[1, 10000] loss: 0.856
[1, 11000] loss: 0.846
[1, 12000] loss: 0.850
[1, 13000] loss: 0.846
[1, 14000] loss: 0.854
[1, 15000] loss: 0.839
[1, 16000] loss: 0.838
[1, 17000] loss: 0.839
[1, 18000] loss: 0.847
[1, 19000] loss: 0.854
[1, 20000] loss: 0.842
[1, 21000] loss: 0.832
[1, 22000] loss: 0.839
[1, 23000] loss: 0.844


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [14]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
losses = []
correct_predictions = 0
n_examples = 0
with torch.no_grad():
    for i, d in enumerate(test_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)
        outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask, labels=labels
        )
        loss = outputs.loss
        logits = outputs.logits
        _, out_amax = torch.max(logits, dim=1)
        n_examples += out_amax.shape[0]
        correct_predictions += torch.sum(out_amax == labels)
        losses.append(loss.item())
        if i % 1000 == 999:
            print(correct_predictions.double() / n_examples, np.mean(losses))

print("Final: ")
print(correct_predictions.double() / n_examples, np.mean(losses))


tensor(0.6516, device='cuda:0', dtype=torch.float64) 0.9076580799669027
tensor(0.6599, device='cuda:0', dtype=torch.float64) 0.8870264119207859
tensor(0.6637, device='cuda:0', dtype=torch.float64) 0.8803491760889689
tensor(0.6675, device='cuda:0', dtype=torch.float64) 0.8688060070052743
tensor(0.6697, device='cuda:0', dtype=torch.float64) 0.8654804260343313
tensor(0.6720, device='cuda:0', dtype=torch.float64) 0.8608454350059231
tensor(0.6738, device='cuda:0', dtype=torch.float64) 0.8563750310391187
tensor(0.6761, device='cuda:0', dtype=torch.float64) 0.8510785044226795
tensor(0.6777, device='cuda:0', dtype=torch.float64) 0.8466681542860137
tensor(0.6798, device='cuda:0', dtype=torch.float64) 0.8425443738982081
Final: 
tensor(0.6798, device='cuda:0', dtype=torch.float64) 0.8425680485905438


In [ ]:
torch.save(model.state_dict(), root_folder+"second_model.pt")

In [ ]:
print(root_folder)

/content/drive/MyDrive/CS182-Spring2020-NLP-Project/
